In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
#function to get the closest matching number from a list K = number to find lst = list to search
def closest(lst, K):
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

#function to draw graph
def getNCalcGraph(fileName=[r"C:\Users\zanyi\OneDrive\Git hub\Python\1284sm.csv",r"C:\Users\zanyi\OneDrive\Git hub\Python\1282.csv"],  #Can add extra files XD
                  savePosition=r"C:\Users\zanyi\OneDrive\Git hub\Python", saveas="testing.png", RoundDigit = 3,IntensityColumn=1, retentionTimeColumn=0):
    fig, ax = plt.subplots()
    saves = []
    #repeat this loop until all file(name/directory up here^^^) is completed
    for names in fileName:
        df = pd.read_csv(names)
        retentionTime = df.iloc[:,retentionTimeColumn].to_list() #get retention time of dataset
        temp = retentionTime[retentionTimeColumn] #should not be important
        rn  = 0  #should not be important
        summsIndex=[] #
        Rtime=[] #
        last=0 #
        # get values in periods of 3000 ms
        for i in range(len(retentionTime)):
            rn = retentionTime[i]
            if rn-temp > 3000:
                temp = retentionTime[i-1]
                Rtime.append(temp)
                summsIndex.append([last,i]) #the indexes of the segments of value (3000ms each rn)
                last = i
        Data = df.iloc[:,IntensityColumn].to_list() #get intensity data from csv
        rsum=[] #sum of each 3000ms segment
        for segment in summsIndex:
            fsum=0
            pts=Data[segment[0]:segment[1]]
            for items in pts:
                fsum+=items
            rsum.append(round(fsum,RoundDigit))

        #cut down the list until 245000 ms later
        cutIndex = Rtime.index(closest(Rtime, 230000))
        cutx = list(Rtime[cutIndex :len(Rtime)-1])
        cuty = list(rsum[cutIndex:len(rsum)-1])
        ax.plot(cutx,cuty, label=f"{Path(names).stem}")
        saves.append(cuty) #save down the cut intensity value

    # ax.set_title('Example plot') #title
    ax.set_xlabel('Retention Time') #x label
    ax.set_ylabel('Values') #y label
    plt.xlim([245000, max(Rtime)]) #set x line
    ax.legend() #show legend
    plt.savefig(savePosition + "/" + saveas) #save graph
    return [saves, cutx] #return intensity value & the retention time value





#get percentage difference from 2 graph & plot them
def getDifference(saves,cutx): 
    log = [] #variable to take down percentage difference
    for ind in range(len(cutx)): #go through all values after 245000ms & calculate difference
        log.append(round((
            abs(saves[1][ind]-saves[0][ind])/abs(saves[0][ind])
            )*100,4))
    fig, ax2 = plt.subplots() #start plotting the difference
    fig, ax3 = plt.subplots()
    for ind,i in enumerate(cutx): #convert milisecond to minutes
        cutx[ind] = i/60000


    #calculating avrg part
    Index30Min = cutx.index(closest(cutx,30)) #Get graph difference before 30 minute (index)
    Index40Min = cutx.index(closest(cutx,40)) #get graph difference after 40 minute (index)
    diffBefore30Min = log[0:Index30Min] #get list values of difference
    diffAfter40Min = log[Index40Min:-1] #get list values of difference
    d30a = sum(diffBefore30Min)/len(diffBefore30Min)
    d40a = sum(diffAfter40Min)/len(diffAfter40Min)
    ax3.plot(cutx[0:Index30Min], diffBefore30Min,color="Green", label = f"Avrg difference before 30 minutes ({d30a})") #plotting
    ax3.plot(cutx[Index40Min:-1], diffAfter40Min,color="blue", label = f"Avrg difference after 40 minutes ({d40a})") #plotting
    ax3.legend(
        loc='upper center', 
        bbox_to_anchor=(0.5,1.2),
        ncol = 6,
        title = "Differences split",
        fontsize = 9.4,
        ) #plotting
    print(f"difference after 40 minutes: {d30a}")
    print(f"difference after 30 minutes: {d40a}") #change to 1 value



    ax2.plot(cutx, log,color="Black", label="differenceOf2Graphs")
    ax2.axhline(y = 20, color = 'red', linestyle = '-', linewidth = 3) #draw 20 line
    ax2.axhline(y = -20, color = 'red', linestyle = '-', linewidth = 3) #draw -20 line
    ax2.grid(axis='y') #draw grid
    ax2.legend(
        loc='upper right', 
        bbox_to_anchor=(1,1.1),
        ncol = 7,
        prop={"size":9}
        ) #plo
    return log,diffBefore30Min,diffAfter40Min #return differance between 2 graphs

getNCalcGraph([r"C:\Users\zanyi\OneDrive\Git hub\Python\1284sm.csv"])#show graph
getNCalcGraph([r"C:\Users\zanyi\OneDrive\Git hub\Python\1282.csv"])#show graph
#getNCalcGraph([r"C:\Users\zanyi\OneDrive\Git hub\Python\1283fid.csv"])#show graph(the other one)
items, cutx = getNCalcGraph()#show combined graph
log,diff30Minute,diff40Minute = getDifference(items,cutx)#show percentage difference graph
totalAvrgdiff = sum(log)/len(log)
print(f"Total avrg difference is: {totalAvrgdiff}")
plt.show()


In [ ]:
#drafting

def decideDifference(time,saves):
    IndexToSave = time.index(closest(time,40)) #get the index of the value after 40 minute
    SavedIndex = 0


    for unit in range(IndexToSave,len(saves[0])): #from index after 40 minute  to   the last item of the list
        SavedIndex += ((saves[1][unit]-saves[0][unit])/saves[0][unit])


    SavedIndex2 = 0
    IndexToSave = time.index(closest(time,30)) #get the index of the value after 30 minute

    for unit2 in range(0,IndexToSave): #from index 0   to   index after 30 minute
        SavedIndex2 += ((saves[1][unit2]-saves[0][unit2])/saves[0][unit2])

    print(f"Percentage difference after 40 minute: {SavedIndex/len(saves[0])*100}")
    print(f"Percentage difference before 30 minute: {SavedIndex2/len(saves[0])*100}")

decideDifference(time=l[1], saves=l[0])


In [ ]:
#lcm 
l=[11,12]
f=0
while True:
    f += min(l)
    if f%max(l)==0:
        print(f)
        break
#hcf
l=[49,56]
s=1
for i in range(1,max(l)+1):
    if min(l)%i==0 and max(l)%i==0:
        s=i
print(s)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import math
def drawGasChromatogram(path = [r"C:\Users\zanyi\OneDrive\Git hub\Python\Outside Work\Data\1284sm.csv",r"C:\Users\zanyi\OneDrive\Git hub\Python\Outside Work\Data\1282.csv"],show=True):
    fig,ax2 = plt.subplots()
    for i in path:
        fig,ax = plt.subplots()
        name = Path(i).stem
        df = pd.read_csv(i)
        ax.plot(df["Time [ms]"],df["Intensity"],label=name)
        ax2.plot(df["Time [ms]"],df["Intensity"],label=name)
        plt.title("Gas Chromatogram Results")
        ax.legend()
        ax2.set_title("Gas Chromatogram Comparison")
    ax2.legend()
    plt.show() if show else None
drawGasChromatogram()

#useless function
def getDiffOfGasChromatogram(path1 = r"C:\Users\zanyi\OneDrive\Git hub\Python\Outside Work\Data\1284sm.csv", path2= r"C:\Users\zanyi\OneDrive\Git hub\Python\Outside Work\Data\1282.csv",show=True):
    df = pd.read_csv(path1)
    df2 = pd.read_csv(path2)
    name1 = Path(path1).stem
    name2 = Path(path2).stem
    newdf=pd.DataFrame()
    newdf["Retention Time"] = (df["Time [ms]"]+df2["Time [ms]"])//2
    newdf["ABSOLUTE Difference compared to path1"] = (abs(df2["Intensity"]-df["Intensity"])/df["Intensity"])*100
    newdf["Difference compared to path1"] = ((df2["Intensity"]-df["Intensity"])/df["Intensity"])*100
    fig, ax = plt.subplots()
    fig, ax2 = plt.subplots()
    ax.plot(newdf["Retention Time"],newdf["ABSOLUTE Difference compared to path1"],label=f"{name1} and {name2} ratio",color="black")
    ax2.plot(newdf["Retention Time"],newdf["Difference compared to path1"],label=f"{name1} and {name2} ratio",color="black")
    ax2.axhline(y = 20, color = 'red', linestyle = '-', linewidth = 3) #draw 20 line
    ax2.axhline(y = -20, color = 'red', linestyle = '-', linewidth = 3) #draw -20 line
    ax.set_title("ABSOLUTE Difference compared to path1")
    ax2.set_title("Difference compared to path1")
    ax.legend()
    ax2.legend()
    plt.show()

getDiffOfGasChromatogram()

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

def button1_action():
    print("Button 1 pressed")
    print_matrix()

def button2_action():
    print("Button 2 pressed")
    print_matrix()

def button3_action():
    print("Button 3 pressed")
    print_matrix()

def choose_directory():
    directory = filedialog.askdirectory()
    if directory:
        print("Selected directory:", directory)
        directory_label.config(text=f"Selected Directory: {directory}")

def print_matrix():
    matrix = [[entries[i][j].get() for j in range(3)] for i in range(3)]
    for row in matrix:
        print(row)

# Create the main window
root = tk.Tk()
root.title("3x3 Matrix Input with Descriptions and Directory Chooser")

# Create a frame for the matrix
matrix_frame = ttk.Frame(root, padding="10")
matrix_frame.grid(row=0, column=1, columnspan=3)

# Create row and column descriptions
row_descriptions = ["Row 1", "Row 2", "Row 3"]
col_descriptions = ["Col 1", "Col 2", "Col 3"]

# Create the 3x3 matrix using Entry widgets with descriptions
entries = [[None for _ in range(3)] for _ in range(3)]

# Add column descriptions
for j in range(3):
    label = ttk.Label(matrix_frame, text=col_descriptions[j])
    label.grid(row=0, column=j+1, padx=5, pady=5)

# Add row descriptions and entry widgets
for i in range(3):
    label = ttk.Label(matrix_frame, text=row_descriptions[i])
    label.grid(row=i+1, column=0, padx=5, pady=5)
    for j in range(3):
        entry = ttk.Entry(matrix_frame, width=5)
        entry.grid(row=i+1, column=j+1, padx=5, pady=5)
        entries[i][j] = entry

# Create the buttons
button1 = ttk.Button(root, text="Button 1", command=button1_action)
button1.grid(row=2, column=0, padx=5, pady=5)

button2 = ttk.Button(root, text="Button 2", command=button2_action)
button2.grid(row=2, column=1, padx=5, pady=5)

button3 = ttk.Button(root, text="Button 3", command=button3_action)
button3.grid(row=2, column=2, padx=5, pady=5)

# Create a button for choosing a directory
choose_dir_button = ttk.Button(root, text="Choose Directory", command=choose_directory)
choose_dir_button.grid(row=3, column=1, padx=5, pady=5)

# Label to display the chosen directory
directory_label = ttk.Label(root, text="No directory selected")
directory_label.grid(row=4, column=1, padx=5, pady=5)

# Start the main event loop
root.mainloop()


In [ ]:
import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def create_plot():
    # Create a figure and axis
    fig = Figure(figsize=(5, 4), dpi=100)
    ax = fig.add_subplot()
    
    # Example data
    x = [0, 1, 2, 3, 4]
    y = [10, 20, 25, 30, 40]
    
    # Plot data
    ax.plot(x, y)
    ax.set_title('Sample Plot')
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    
    return fig

def main():
    # Create the main Tkinter window
    root = tk.Tk()
    root.title("Matplotlib Plot in Tkinter")

    # Create a plot and get the figure
    fig = create_plot()

    # Create a canvas widget to embed the Matplotlib figure
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()

    # Get the canvas widget and place it in the Tkinter window
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    # Add a button to exit the application
    button = tk.Button(master=root, text="Quit", command=root.quit)
    button.pack(side=tk.BOTTOM)

    # Start the Tkinter event loop
    root.mainloop()

if __name__ == "__main__":
    main()



In [1]:
# Import required libraries
from tkinter import *
from PIL import ImageTk, Image
import os,sys

# Create an instance of tkinter window
win = Tk()

# Define the geometry of the window
width= win.winfo_screenwidth() 
height= win.winfo_screenheight()
win.geometry("%dx%d" % (width, height))

# #setup image gallery
# frame = Frame(win, width=10, height=10)
# frame.grid()
# frame.place(anchor='center', relx=0.6, rely=0.5)

# # Create an object of tkinter ImageTk
# size = width,height

# try:
#     photo = Image.open(r"C:\Users\zanyi\OneDrive\Git hub\Python\Outside Work\Done\BarGraph2.png")
#     photo.thumbnail(size, Image.Resampling.LANCZOS)
# except IOError:
#     print("cannot create")
# img = ImageTk.PhotoImage(photo)

# # Create a Label Widget to display the text or Image
# label = Label(frame, image = img)
# label.pack()
test = Image.open(r"C:\Users\zanyi\OneDrive\Git hub\Python\Outside Work\Done\BS10LineGraph.png")
test = ImageTk.PhotoImage(test)
frame = Frame(win, width=10, height=10)
frame.grid()
frame.place(anchor='center', relx=0.6, rely=0.5)
size=width,height
label = Label(frame,image=test)
label.grid()
win.mainloop()

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import os

def load_images():
    global images, image_files
    
    # Open file dialog to select a directory
    
    # Get list of image files in the directory
    image_files = [r"C:\Users\zanyi\OneDrive\Git hub\Python\Outside Work\Done\BarGraph.png"]
    
    # Load images using PIL
    images = [Image.open(img_file) for img_file in image_files]

def show_image(index):
    global image_label, current_image_index
    
    # Get the image from the list and resize it
    img = images[index]
    img = img.resize((400, 400), LANCZOS )
    img_tk = ImageTk.PhotoImage(img)
    
    # Update the image label with the new image
    image_label.config(image=img_tk)
    image_label.image = img_tk

def show_next_image():
    global current_image_index
    
    # Increment the image index
    current_image_index += 1
    
    # Wrap around if the index exceeds the number of images
    if current_image_index >= len(images):
        current_image_index = 0
    
    # Show the next image
    show_image(current_image_index)

def show_prev_image():
    global current_image_index
    
    # Decrement the image index
    current_image_index -= 1
    
    # Wrap around if the index is less than zero
    if current_image_index < 0:
        current_image_index = len(images) - 1
    
    # Show the previous image
    show_image(current_image_index)

def main():
    global image_label, current_image_index, images, image_files
    
    # Initialize global variables
    images = []
    image_files = []
    current_image_index = 0

    # Create the main Tkinter window
    root = tk.Tk()
    root.title("Image Gallery")

    # Frame for image display
    image_frame = tk.Frame(root)
    image_frame.pack()

    # Label to display images
    image_label = tk.Label(image_frame)
    image_label.pack()

    # Load images
    load_images()

    # Display the first image
    if images:
        show_image(current_image_index)

    # Buttons for navigation
    prev_button = tk.Button(root, text="Previous", command=show_prev_image)
    prev_button.pack(side=tk.LEFT)

    next_button = tk.Button(root, text="Next", command=show_next_image)
    next_button.pack(side=tk.RIGHT)

    # Start the Tkinter event loop
    root.mainloop()

if __name__ == "__main__":
    main()
